In [4]:
from google.colab import drive
drive.mount('/content/drive')
!cd gdrive/My\ Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/bin/bash: line 1: cd: gdrive/My Drive: No such file or directory


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding
from tensorflow.keras.layers import Conv1D,Dropout,SpatialDropout1D,MaxPooling1D,GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
df=pd.read_csv("toxic_comments_classification_data.csv")
df.head()

,comment_text,toxic
0,This letter perfectly illustrates why any hope...,1
1,One muslim casualty vs the hundreds and thousa...,1
2,(fuck you Osama bin laden and your afghanistan...,1
3,As long as Trump keeps Stiggin' It to the libs...,1
4,This article is a load of crap.... Another Fa...,1


In [7]:
df.isnull().sum()

,0
comment_text,0
toxic,0


In [8]:
df.duplicated().sum()

27

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
x=df['comment_text'].to_list()
y=df["toxic"]

In [11]:
stop_words=stopwords.words('english')
stemmer=PorterStemmer()

x_train=[]
for sentence in x:
  sentence=re.sub("[^a-zA-Z]"," ",sentence)
  sentence=sentence.lower().split()
  sentence=[stemmer.stem(word) for word in sentence if word not in stop_words]
  sentence=" ".join(sentence)
  x_train.append(sentence)

In [12]:
x_train[:5]

['letter perfectli illustr hope reconcili sheer stupid',
 'one muslim casualti vs hundr thousand victim muslim terror year alon darn islamophobia',
 'fuck osama bin laden afghanistani terrorist cunt',
 'long trump keep stiggin lib palin american care healthcar damn full steam ahead',
 'articl load crap anoth fake news poll']

In [13]:
tokenizer=Tokenizer(num_words=20000)
tokenizer.fit_on_texts(x_train)

In [14]:
vocab_size=len(tokenizer.word_index)+1
vocab_size

33205

In [15]:
encode_texts=tokenizer.texts_to_sequences(x_train)

In [16]:
#max_len_list=[len(i.split(" ")) for i in x_train]
#max_len=max(max_len_list)
#max_len
max_len = min(200, max([len(i.split(" ")) for i in x_train]))
max_len

200

In [17]:
padded_x=pad_sequences(encode_texts,maxlen=max_len,padding="post",truncating="post")

In [18]:
x_train,x_test,y_train,y_test=train_test_split(padded_x,y,test_size=0.15,random_state=42)

In [19]:
embedding_features=300

In [20]:
model=Sequential()
model.add(Embedding(vocab_size,embedding_features,input_length=max_len,))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(64,recurrent_activation="sigmoid",dropout=0.2,return_sequences=True))
model.add(LSTM(32,recurrent_activation="sigmoid",dropout=0.2))
model.add(Dense(1,activation="sigmoid"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [21]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [22]:
history=model.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 332s 612ms/step - accuracy: 0.4997 - loss: 0.6948 - val_accuracy: 0.4977 - val_loss: 0.6930
Epoch 2/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 388s 623ms/step - accuracy: 0.5052 - loss: 0.6937 - val_accuracy: 0.4977 - val_loss: 0.6930
Epoch 3/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 374s 608ms/step - accuracy: 0.5065 - loss: 0.6901 - val_accuracy: 0.5117 - val_loss: 0.6891
Epoch 4/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 379s 602ms/step - accuracy: 0.6264 - loss: 0.5927 - val_accuracy: 0.9126 - val_loss: 0.2487
Epoch 5/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 324s 611ms/step - accuracy: 0.9349 - loss: 0.1998 - val_accuracy: 0.9196 - val_loss: 0.2235
Epoch 6/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 322s 606ms/step - accuracy: 0.9605 - loss: 0.1284 - val_accuracy: 0.9222 - val_loss: 0.2441
Epoch 7/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 378s 599ms/step - accuracy: 0.9805 - loss: 0.0707 - val_accuracy: 0.9192 - val_loss: 0.2881
Epoch 8/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 317s 596ms/step - accuracy: 0.9871 -

In [29]:
def text_processing(text):
  encoded=tokenizer.texts_to_sequences(text)
  padded=pad_sequences(encoded,maxlen=max_len,padding="post",truncating="post")
  return padded
comment=["Non-Toxic","Toxic"]

In [31]:
tweet = ["Hello, have a good day"] #neg sentiment 0
txt = text_processing(tweet)
output = comment[int(round(model.predict(txt)[0][0]))]
print(output)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Non-Toxic
